# Problème

Nous devons classifier des images de chiffres entre 0 et 9.

Notre data set est formaté comme suis : Label | Pixel 0 | Pixel 1 | ... | Pixel 783

Notre réseau prendra alors la forme de : Pixel Layer | Hidden layers | Digit layer

Le digit layer est composé de 10 neuronnes représentant chacun le pourcentage de reconnaissance d'un chiffre.

Le pixel layer sera composé des pixels de l'image.

Le nombre de hidden layer et leur densité sera évaluée en fonction des digit layer et pixel layer.

Pour entrainer et tester notre réseau, nous avons à disposition 3 fichiers

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from tensorflow import keras
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

# Récupération des données

Nous récupérons ici nos données d'entrainement et de test.

Les labels sont contenus à l'index ["label"] des données d'entrainement.

Il nous faut donc supprimer cet index pour récupérer seulement les pixels

In [ ]:
full_train_set = pd.read_csv("../input/train.csv")
final_set = pd.read_csv("../input/test.csv")

print(full_train_set.shape)

full_labels = full_train_set["label"]
full_pixels = full_train_set.drop(labels = ["label"], axis = 1)

train_labels, test_labels, train_pixels, test_pixels = train_test_split(full_labels, full_pixels)

print("Train Labels :", train_labels.shape)
print("Train Pixels :", train_pixels.shape)

print("Test Labels :", test_labels.shape)
print("Test Pixels :", test_pixels.shape)

print("Final Pixels :", final_set.shape)

# Normalisation

Les données sont ensuite normalisées entre 0 et 1 pour pouvoir être utilisées dans un réseau

In [ ]:
train_pixels = train_pixels / 255.0
test_pixels = test_pixels / 255.0
final_set = final_set / 255.0

print(train_pixels.shape)
print(test_pixels.shape)
print(final_set.shape)

# Remodelage des données

Nous transformons le tableau de labels en matrice par index : ex :  [0, 2, 1] => [[1, 0, 0], [0, 0, 1], [0, 1, 0]]

Nous remodelons le tableau unique de 784 pixels en un tableau à double entrée de 28x28px

In [ ]:
train_labels = keras.utils.to_categorical(train_labels, num_classes = 10)
train_pixels = train_pixels.values.reshape(-1, 28, 28, 1)

test_labels = keras.utils.to_categorical(test_labels, num_classes = 10)
test_pixels = test_pixels.values.reshape(-1, 28, 28, 1)

final_set = final_set.values.reshape(-1, 28, 28, 1)

print("Train Pixels: ", train_pixels.shape)
print("Test Pixels: ", test_pixels.shape)
print("Final Pixels: ", final_set.shape)

# Affichage

Si nous affichons une image du tableau, on remarque que les données sont bien formatées

In [ ]:
plt.imshow(train_pixels[50][:,:,0])

In [ ]:
model = keras.Sequential()

#réseau de convolution
model.add(keras.layers.Conv2D(filters=32, kernel_size=(5,5), padding='same', activation='relu', input_shape=(28,28,1)))
model.add(keras.layers.Conv2D(filters=32, kernel_size=(5,5), padding='same', activation='relu', input_shape=(28,28,1)))
model.add(keras.layers.MaxPool2D(pool_size=(2,2)))
model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Conv2D(filters=64, kernel_size=(2, 2), padding='same', activation='relu', input_shape=(28, 28, 1)))
model.add(keras.layers.Conv2D(filters=64, kernel_size=(2, 2), padding='same', activation='relu', input_shape=(28, 28, 1)))
model.add(keras.layers.MaxPool2D(pool_size=(2,2)))
model.add(keras.layers.Dropout(0.25))

#layer d'applatissement de l'image en 1 dimension
model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(256))
model.add(keras.layers.Dropout(0.5))

#layer d'output
model.add(keras.layers.Dense(10, activation='softmax'))

#compilation du modèle avec une descente de gradient et la méthode des moindres carrés
model.compile(optimizer = "Adam", loss = "categorical_crossentropy", metrics=["accuracy"])

model.fit(train_pixels, train_labels, epochs=20, batch_size=64)

# Evaluate the model

Nous évaluons ensuite le modèle avec l'ensemble de test que nous avons découpé au début.

Cette étape nous permettra d'obtenir une accuracy et une loss sur un jeu de données complètement inconnu du modèle

In [ ]:
loss, acc = model.evaluate(test_pixels, test_labels, batch_size=64)

print('Loss: ', loss)
print('Accuracy: ', acc)

# Affichage du résultat

Afin de nous rendre compte nous même du résultat, nous affichons les prédicitions sur les 25 premiers éléments de notre jeu de données final et on remarque une très bonne précision.

In [ ]:
limit = 25
width = 5
height = int(limit / width)

fig, axis = plt.subplots(height, width, sharex=True, sharey=True, figsize=(width * 5, height * 5))

final_labels = model.predict(final_set[:limit])

for i in range(limit):
    row = int(i / width)
    col = int(i % width)
    axis[row,col].imshow(final_set[i][:,:,0])
    axis[row,col].set_title("Label :{}\n".format(np.argmax(final_labels[i])))
    axis[row,col].title.set_fontsize(20)

plt.show()

#plt.imshow(test_set[0][:,:,0])
#print(np.argmax(test_labels[:10,0]))

# Publication des résultats

Il nous faudra finalement publier nos résultats dans un csv au format donné dans le sujet

In [ ]:
final_labels = np.argmax(model.predict(final_set), axis=1).reshape((-1, 1))
indexes = [[i] for i in range(1, len(final_labels) + 1)]
data = np.concatenate((indexes, final_labels), axis=1)

results = pd.DataFrame(data=data, columns=['ImageId','Label'])

results.to_csv('submission.csv', index=False)

In [ ]:
print(os.listdir("."))